In [1]:
#implort Libraries:
import pandas as pd
import numpy as np
import json
import os
import io
import subprocess
import random
import logging
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from sklearn.metrics import accuracy_score
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFSyntaxError

In [2]:
import nltk
from nltk.corpus import stopwords
import spacy
from string import punctuation
import docx2txt
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\azerty\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\azerty\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [37]:
def convert_annotation(df):
    train_data = []
    for index, row in df.iterrows():
        content = convertTopdf(PATH_PDF + '/' + row.file).lower()
        content = " ".join(content.split())
        print(row.file)
        entities = extract_info(content, row.text.lower())
        train_data.append((content, {"entities":entities}))
    return train_data

In [51]:
PATH_WORD= "C:/Users/azerty/Downloads/Andex/Word"
PATH_PDF= "C:/Users/azerty/Downloads/Andex/PDF"
# create blank Language class
nlp = spacy.blank('fr')

In [52]:
listpdf = os.listdir(PATH_PDF)
listword = os.listdir(PATH_WORD)
listfiles = listword + listpdf

In [53]:
#convert files pdf to text:
def convertTopdf(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = io.StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text 

In [54]:
#convert files docx to text:
def convertTodocx(doc_path):
    try:
        temp = docx2txt.process(doc_path)
        text = temp.replace('\t', '')
        return ''.join(text)
    except KeyError:
        return ' '

In [55]:
def convertTodoc(filepath, file):
    doc_file = filepath + file
    docx_file = filepath + file + 'x'
    if not os.path.exists(docx_file):
        print('antiword ' + doc_file + ' > ' + docx_file)
        os.system('antiword ' + doc_file + ' > ' + docx_file)
        with open(docx_file) as f:
            text = f.read()
            print(text)
        os.remove(docx_file) #docx_file was just to read, so deleting
    else:
        print('Info : file with same name of doc exists having docx extension, so we cant read it')
        text = ''
    return text

In [56]:
def find_index(content, text):
    start = content.find(text)
    end = len(text) + start
    return start, end

In [57]:
def extract_info(content, t):
    couples = t.split(';;;')
    info = []
    for couple in couples:
        text, label = couple.split(':::')
        start=content.find(text[:50]) 
        end=start+len(text) 
        pair = (start, end, label)
        info.append(pair)
    return info

In [61]:
#Data annotation:
def convertion(df):
    train_data = []
    for index, row in df.iterrows():
        content = convertTopdf(PATH_PDF + '/' + row.file).lower()
        content = " ".join(content.split())
        entities = extract_info(content, row.text.lower())
        train_data.append((content, {"entities":entities}))
    print("conversion reussite")
    return train_data

In [62]:
train = pd.read_excel('C:/Users/azerty/Desktop/5DS/PROJET NLP/train.xlsx', header=0)




In [63]:
Train =convertion(train)

conversion reussite


In [64]:
#add list Labels:
LABELS = ['job title','ville','missions','experience','contrat','formation']

### spacy-ner-annotator


In [69]:
 # nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)

In [70]:
  # add labels
for _, annotations in Train:
    for ent in annotations.get('entities'):
        for label in LABELS:
            ner.add_label(label)

In [71]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

In [72]:
# get names of other pipes to disable them during training
def train(Train, nlp):    
    l = []
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        los=501
        for itn in range(100):
        
            random.shuffle(Train)
            losses = {}
            
            if(los>500):
                for text, annotations in Train:
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.01,  # dropout - make it harder to memorise data
                        sgd=optimizer,  # callable to update weights
                        losses=losses)
                
                los=int(losses.get('ner'))
                l.append([itn, losses.get('ner')])
                
            else:
                return l
            
    return l

In [86]:
def test(files):
    df=pd.DataFrame(columns=["MissionsCOMPETENCES"])
    i_files=0
    for f in files:
        c = ''
        i_files=i_files+1
    
        if 'docx' in f:
            c = convertTodocx(PATH_WORD + '/' + f).lower().replace('\n', '')
        elif 'pdf' in f:
            c = convertTopdf(PATH_PDF + '/' + f).lower().replace('\n', '')
        else:
            next
        doc2 = nlp(c)
        for ent in doc2.ents:
            if(ent.label_=="missions"):
                df.loc[i_files]=ent.text         
    return df  

In [87]:
TrainedModel = train(Train, nlp)

In [88]:
outputFiles=test(files)

In [89]:
outputFiles.head(3)

,MissionsCOMPETENCES
1, gérer la contractualisation (avec les clien...
2,rédiger les contrats et avenants (en anglais e...
4,gérer la relation commerciale d'un ou plusieur...


In [90]:
l=outputFiles.values.tolist()

In [91]:
output=outputFiles


In [92]:
(output[0:1].MissionsCOMPETENCES)

1      gérer la contractualisation (avec les clien...
Name: MissionsCOMPETENCES, dtype: object

In [93]:
a=str(l[0])

In [94]:
l[3]

['• apportez un support quotidien sur la rédaction et négociation des contrats, incluant notamment les contrats de tenue de compte - conservation, les conventions dépositaire, les contrats de réception transmission d\'ordres sur opc, les contrats de gestion comptable, les contrats de correspondant centralisateur, les contrats \x0cd\'échange d\'information et, dans ce cadre, vous réalisez toutes taches afférentes à ces négociations ;          • vous effectuez une veille juridique et règlementaire, essentiellement dans le domaine du droit bancaire et financier ;        •  vous rédigez des notes juridiques à destination des équipes commerciales et opérationnelles.      l\'environnement de travail, c\'est important ! le poste est à pourvoir en cdd dans le cadre d\'un remplacement de congé maternité.  vous intégrez une équipe juridique de 5 personnes dédiée aux lignes clients « asset managers » et « asset owners ». rattaché(e) au responsable juridique de l\'équipe, vous travaillez sur les p

### Dictionaire Semantique

In [99]:
Dic={
    "Droit français":["droit des contrats","droit commercial","droit des garanties","droit fiscal des entreprises",
                      "droit patrimonial","régime matrimonial","succession","droit bancaire"],
    "Droit bancaire":["FBF","ISDA","Normes comptables","IFRS","AAP","Droit financier","code monétaire et financier", "CMF"],
    "Produits bancaires":["produits dérivés", "instruments financiers" , "produit d’investissement",
"produits de placement","produit de spéculation","instrument de marchés","produits structurés","produits d’épargne" , "produits d’assurance","émissions obligataires" , "titres de créance",
"Opérations de marché","Marchés financiers"],
    "Fiscalité":["l’impôt sur le revenu" , "IR" ,"l'impôt sur les sociétés" ,"IS","impôts sur les bénéfices",
"l’impôt de solidarité sur la fortune","ISF","la taxe sur la valeur ajoutée","TVA","la taxe intérieure sur les produits pétroliers",
                 "TIPP","code général des impôts","CGI","taxes","CGS","CRDS"],
    "Risque bancaire":["risque de marché","risque opérationnel", "risque de contrepartie","risque de crédit",
        "Normes de réglementation","FRTB","BALE2","BALE 2.5","EMIR","VAR","LCR","CVAR","Expected Shortfall","ES"],
    "Outils informatiques":["Microsoft Office","Excel","Outlook","Access","VBA","Word","bureautique"],
    "Organisation":["Agenda","Rigueur","planifier","Travail en équipe","Outlook"],
    "Communication":["Convaincre","Relationnel","collaboration","réunions","compte rendu","Anglais Ecrit" , "Oral"]
        
}

In [104]:
def getAllCompetences(Dic):
    allCompetences=[]
    for cle, valeur in Dic.items():
        for i in range(0,len(valeur)-1):
            allCompetences.append(valeur[i])
    return list(dict.fromkeys(allCompetences)) 

In [95]:
#Metrique de similarité

In [105]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [106]:
competencesCle=[]
l=[]
c=[]
for index, row in output.iterrows():
    ligne=row["MissionsCOMPETENCES"]
    
    #listeCompetence=[]
   
    competences=[]
    tokens = nltk.word_tokenize(str(ligne))
    for cle, valeur in Dic.items():

        for i in range(0,len(valeur)-1):
            listeCompetence=[]
            for j in range(0,len(tokens)-1):
                #calculer la similarite: 
                if(similar(tokens[j].lower(),valeur[i].lower())>0.66):
                    #listeCompetence.append(valeur[i])
                    competences.append(valeur[i])
    l.append(ligne)
    c.append(competences)
    #df=transformer(output,competencesCle,Dic)
#return df
competencesCle=dict(zip(l,c))

In [107]:
liste=list(getAllCompetences(Dic))
l=[]
lo=[]
onehotencoding=[]
for cle,v in competencesCle.items():
        find=[0]*len(liste)
        for j in v:
            l.append(liste.index(j))
        for i in range(0,len(l)-1):
            find[l[i]]=1
        #lo.append([cle,find])
        onehotencoding.append(find)

In [108]:
gg=onehotencoding

In [109]:
gg=np.array(onehotencoding)
listeNew=gg.transpose( )

In [112]:
d = dict.fromkeys(liste, 0)
newdf=output.assign(**d)


In [114]:
newdf[liste]=onehotencoding

In [115]:
newdf

,MissionsCOMPETENCES,droit des contrats,droit commercial,droit des garanties,droit fiscal des entreprises,droit patrimonial,régime matrimonial,succession,FBF,ISDA,...,Agenda,Rigueur,planifier,Travail en équipe,Convaincre,Relationnel,collaboration,réunions,compte rendu,Anglais Ecrit
1, gérer la contractualisation (avec les clien...,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,rédiger les contrats et avenants (en anglais e...,0,0,0,0,0,0,1,0,1,...,0,1,0,0,0,1,0,1,0,0
4,gérer la relation commerciale d'un ou plusieur...,0,1,0,0,0,0,1,0,1,...,0,1,0,0,0,1,0,1,0,1
11,• apportez un support quotidien sur la rédacti...,0,1,0,0,0,0,1,0,1,...,0,1,0,0,1,1,0,1,0,1
17,son activité est de traiter toutes les questio...,0,1,0,0,0,0,1,0,1,...,0,1,0,0,1,1,0,1,1,1
18,"élaborer les modèles de règlements, d’accords,...",0,1,0,0,0,0,1,0,1,...,0,1,0,0,1,1,1,1,1,1
19,"conseiller et assister le secrétariat général,...",0,1,0,0,0,0,1,0,1,...,0,1,0,0,1,1,1,1,1,1
23,rédiger et négocier la documentation juridique...,0,1,0,0,0,0,1,0,1,...,0,1,0,0,1,1,1,1,1,1
24,d’expérience,0,1,0,0,0,0,1,0,1,...,0,1,0,0,1,1,1,1,1,1
36,compréhension des enjeux et des métiers des pé...,0,1,0,0,0,0,1,0,1,...,0,1,0,0,1,1,1,1,1,1


In [116]:
newdf.to_csv("final_output_andex.csv")